In [2]:
import sys
import platform
print("Python version:\n", sys.version)
print ("Path to the python executable:\n", sys.executable)

Python version:
 3.6.10 |Anaconda, Inc.| (default, May  7 2020, 19:46:08) [MSC v.1916 64 bit (AMD64)]
Path to the python executable:
 C:\Users\user\anaconda3\envs\py36\python.exe


In [3]:
import tensorflow as tf 
import datetime
import time
import os
from IPython.display import Image

import numpy as np

import settings    # Use the custom settings.py file for default parameters
# import onnxmltools

import sys; sys.argv=['']; del sys
# from model import load_model, get_callbacks, evaluate_model
from data import load_data
from model import unet

from argparser import args

if args.keras_api:
    import keras as K
else:
    from tensorflow import keras as K

print ("We are using Tensorflow version", tf.__version__,\
       "with Intel(R) MKL", "enabled" if tf.pywrap_tensorflow.IsMklEnabled() else "disabled",)
print("Keras API version: {}".format(K.__version__))

onnx = False # Set whether we are exporting to ONNX model and using nGraph

We are using Tensorflow version 1.15.0 with Intel(R) MKL disabled
Keras API version: 2.3.1


Using TensorFlow backend.


In [4]:
#TODO: Set the multi-threading parameters for Tensorflow. 
config = tf.ConfigProto(intra_op_parallelism_threads=args.num_threads,
                        inter_op_parallelism_threads=args.num_inter_threads)

sess = tf.Session(config=config)

K.backend.set_session(sess)

In [5]:
hdf5_filename = os.path.join(args.data_path, args.data_filename)
print(args.data_path)
print ("Full path to the HFS file:", hdf5_filename)

data/decathlon/
Full path to the HFS file: data/decathlon/Task01_BrainTumour.h5


In [6]:
imgs_train, msks_train, imgs_validation, msks_validation, imgs_testing, msks_testing = \
    load_data(hdf5_filename, args.batch_size,[args.crop_dim, args.crop_dim])

np.random.seed(816)

Batch size = 128
Training image dimensions:   (58464, 144, 144, 4)
Training mask dimensions:    (58464, 144, 144, 1)
Validation image dimensions: (4608, 144, 144, 4)
Validation mask dimensions:  (4608, 144, 144, 1)
Testing image dimensions: (6624, 144, 144, 4)
Testing mask dimensions:  (6624, 144, 144, 1)


In [7]:

print("-" * 30)
print("Creating and compiling model ...")
print("-" * 30)
unet_model = unet()
model = unet_model.create_model(imgs_train.shape, msks_train.shape)

model_filename, model_callbacks = unet_model.get_callbacks()

# # If there is a current saved file, then load weights and start from there.
# saved_model = os.path.join(args.output_path, args.inference_filename)
# if os.path.isfile(saved_model):
#     model.load_weights(saved_model)

------------------------------
Creating and compiling model ...
------------------------------
Data format = channels_last
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Using Transposed Deconvolution


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
MRImages (InputLayer)           (None, None, None, 4 0                                            
__________________________________________________________________________________________________
encodeAa (Conv2D)               (None, None, None, 3 1184        MRImages[0][0]                   
__________________________________________________________________________________________________
encodeAb (Conv2D)               (None, None, None, 3 9248        encodeAa[0][0]                   
_____________________________________________________

In [8]:
print("-" * 30)
print("Fitting model with training data ...")
print("-" * 30)

print("Step 3, training the model started at {}".format(datetime.datetime.now()))
start_time = time.time()

if onnx:
    for i in range(0,30):
        print ("Processing batch:", i)
        start_time = time.time()
        model.train_on_batch(imgs_train[i*args.batch_size:args.batch_size*(i+1)-1], \
                             msks_train[i*args.batch_size:args.batch_size*(i+1)-1])
        print ("Time for training on batch:", time.time() - start_time) 

        # TODO: Convert the Keras model to ONNX and save it. 
        onnx_model = onnxmltools.convert_keras(model, target_opset=7) 
        onnxmltools.utils.save_model(onnx_model, 'output/unet_model_for_decathlon.onnx')
else:
    history = model.fit(imgs_train, msks_train,
              batch_size=args.batch_size,
              epochs=args.epochs,
              validation_data=(imgs_validation, msks_validation),
              verbose=1, shuffle="batch",
              callbacks=model_callbacks)
    
print("Total time elapsed for training = {} seconds".format(time.time() - start_time))
print("Training finished at {}".format(datetime.datetime.now()))
    
# Append training log
# with open("training.log","a+") as fp:
#     fp.write("{}: {}\n".format(datetime.datetime.now(),
#                              history.history["val_dice_coef"]))

------------------------------
Fitting model with training data ...
------------------------------
Step 3, training the model started at 2020-05-21 10:46:45.792166


KeyboardInterrupt: 

In [9]:
print("-" * 30)
print("Loading the best trained model ...")
print("-" * 30)
unet_model.evaluate_model(model_filename, imgs_testing, msks_testing)

------------------------------
Loading the best trained model ...
------------------------------

Evaluating model on test dataset. Please wait...
6624/6624 [==============================] - 1406s 212ms/step
6624 images in 1406.27 seconds => 4.710 images per second inference
Test dataset loss = 0.2276
Test dataset accuracy = 0.9885
Test dataset dice_coef = 0.8054
Test dataset soft_dice_coef = 0.6570
